In [6]:
sentences = [
    "Была госпитализирована в Тоскенскую ЦРБ с диагнозом ОИМ.",
    "18.08.11 обратилась в поликлинику с жалобами на длительные жгучие боли в области сердца, одышку.",
    'На фоне проводимого лечения сохранялись боли в области сердца в связи с чем была направлена на госпитализацию в федеральное государственное бюджетное учреждение "Федеральный центр сердца крови и эндокринологии им Алмазова".',
]

In [1]:
import re

with open("ru_syntagrus-ud-test.conllu", "r") as f:
    text = f.read()

pattern = "text = (.*?)\n1"
test_sentences = re.findall(pattern, text)

In [1]:
import pathlib

sentences_7777 = list()
dataset_file = pathlib.Path(r"Data/separeted_sentences_txt_extra_7777")
docs = [file for file in dataset_file.iterdir() if file.suffix == ".txt"]
docs.sort()

for doc in docs:
    with open(doc, 'r') as f:
        text = f.read()
        text = text.replace('\n', '')
        sentences_7777.append(text)

# qbic

https://github.com/AlexeySorokin/GramEval2020

## Установка необходимых библиотек

In [ ]:
!git clone https://github.com/DanAnastasyev/GramEval2020.git
!cd GramEval2020/ && pip install -r requirements.txt && ./download_data.sh
!pip install git+git://github.com/DanAnastasyev/allennlp.git

In [ ]:
!cd GramEval2020 \
    && ./download_model.sh trainable_bert_morph_lstm \
    && cd solution \
    && python -m train.applier --model-name trainable_bert_morph_lstm --batch-size 8

## Test

Необходимо поменять пути до нужных файлов

In [ ]:
%%time
!cd GramEval2020/solution \
    && python -m train.applier \
        --pretrained-models-dir /home/renoip/Jupyter/Parsers/qbic/GramEval2020/pretrained_models \
        --models-dir /home/renoip/Jupyter/Parsers/qbic/GramEval2020/models \
        --model-name trainable_bert_morph_lstm --batch-size 8 \
        --data-dir /home/renoip/Jupyter/Parsers/qbic/Data \
        --predictions-dir /home/renoip/Jupyter/Parsers/qbic/Preds

Time - 1m 58s (GPU)

In [7]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu qbic_test.conllu

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     98.79 |     98.37 |     98.58 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |     98.79 |     98.37 |     98.58 |
UPOS       |     97.64 |     97.22 |     97.43 |     98.83
XPOS       |     98.79 |     98.37 |     98.58 |    100.00
UFeats     |     95.00 |     94.60 |     94.80 |     96.17
AllTags    |     94.16 |     93.76 |     93.96 |     95.32
Lemmas     |     97.68 |     97.26 |     97.47 |     98.88
UAS        |     97.18 |     96.77 |     96.98 |     98.38
LAS        |     94.36 |     93.96 |     94.16 |     95.52
CLAS       |     92.84 |     92.62 |     92.73 |     94.36
MLAS       |     86.80 |     86.59 |     86.69 |     88.22
BLEX       |     91.56 |     91.33 |     91.45 |     93.05


* UAS - 96.98
* LAS - 94.16

использовал токенизацию от Slovnet

# Turku

http://turkunlp.org/Turku-neural-parser-pipeline/

Можно установить используя docker контейнер http://turkunlp.org/Turku-neural-parser-pipeline/docker.html

Time - 5min (CPU)

In [15]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu Turku_test.txt

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     99.60 |     99.60 |     99.60 |
Sentences  |     97.85 |     98.26 |     98.06 |
Words      |     99.60 |     99.60 |     99.60 |
UPOS       |     97.72 |     97.73 |     97.72 |     98.12
XPOS       |     99.60 |     99.60 |     99.60 |    100.00
UFeats     |     96.19 |     96.20 |     96.20 |     96.59
AllTags    |     95.43 |     95.43 |     95.43 |     95.82
Lemmas     |     94.30 |     94.31 |     94.30 |     94.68
UAS        |     88.86 |     88.87 |     88.86 |     89.22
LAS        |     87.08 |     87.09 |     87.08 |     87.43
CLAS       |     89.78 |     89.65 |     89.71 |     90.08
MLAS       |     84.91 |     84.79 |     84.85 |     85.20
BLEX       |     83.03 |     82.91 |     82.97 |     83.31


* UAS - 88.86
* LAS - 87.08

Стоит отметить, что парсер сам выполнял разбиение на предложения, в отличии от готового разбиения в других парсерах.

# Spacy

https://github.com/buriy/spacy-ru

## Установка необходимых библиотек

Для модели spacy для русского языка требуется pymorphy2 версии 0.8 и spacy версии 2.1.9.

In [ ]:
!pip install pymorphy2==0.8

In [ ]:
!pip install spacy==2.1.9

In [ ]:
!git clone -b v2.1 https://github.com/buriy/spacy-ru.git && cp -r ./spacy-ru/ru2/.

In [ ]:
!pip install conllu

## Построение деревьев

In [2]:
from nltk import Tree
import spacy

nlp = spacy.load("spacy-ru/ru2")

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [ ]:
%%time
for sent in sentences:
    doc = nlp(sent)
    print(sent, end="\n\n")
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]
    print("*" * 150)

## Сохранение результата в формате Conll-U

In [6]:
from spacy_conll import ConllFormatter

nlp = spacy.load("spacy-ru/ru2")
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter, after='parser')

In [ ]:
%%time
with open("Spacy_7777.txt", "w") as f:
    for sent in range(len(sentences_7777)):
        doc = nlp(sentences_7777[sent])
        sent_id = str(docs[sent])
        print("# sent_id = " + sent_id[sent_id.rfind("/") + 1 :], file=f)
        print("# text = " + sentences_7777[sent], file=f)
        for i in range(len(doc)):
            token = doc[i]
            if token.dep_ =='root':
                dep = 0
            else: dep = token.head.i+1
            print('{}\t{}\t{}\t{}\t_\t_\t{}\t{}\t_\t_'.format(i+1, token.text, token.lemma_.strip(), token.pos_, dep, token.dep_), file=f)
        print(file=f)

## Test

In [ ]:
%%time
with open("Spacy_test.txt", "w") as f:
    for sent in range(len(test_sentences)):
        doc = nlp(test_sentences[sent])

        print("# sent_id = " + str(sent), file=f)
        print("# text = " + test_sentences[sent], file=f)
        for i in range(len(doc)):
            token = doc[i]
            if token.dep_ =='root':
                dep = 0
            else: dep = token.head.i+1
            print('{}\t{}\t{}\t{}\t_\t_\t{}\t{}\t_\t_'.format(i+1, token.text, token.lemma_.strip(), token.pos_, dep, token.dep_), file=f)
        print(file=f)

Time - 2min 39s (CPU)

In [8]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu Spacy_test.txt

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     97.91 |     98.70 |     98.31 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |     97.91 |     98.70 |     98.31 |
UPOS       |     95.39 |     96.16 |     95.77 |     97.42
XPOS       |      0.00 |      0.00 |      0.00 |      0.00
UFeats     |     91.38 |     92.12 |     91.75 |     93.33
AllTags    |      0.00 |      0.00 |      0.00 |      0.00
Lemmas     |     86.58 |     87.28 |     86.93 |     88.42
UAS        |     83.12 |     83.80 |     83.46 |     84.90
LAS        |     75.26 |     75.87 |     75.56 |     76.86
CLAS       |     81.48 |     74.43 |     77.80 |     75.60
MLAS       |     73.84 |     67.45 |     70.50 |     68.51
BLEX       |     68.71 |     62.77 |     65.61 |     63.75


* UAS - 86.93
* LAS - 75.56

# DeepPavlov

http://docs.deeppavlov.ai/en/master/features/models/syntaxparser.html

## Установка необходимых библиотек

In [ ]:
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install syntax_ru_syntagrus_bert

## Сохранение результата в формате Conll-U

Не чувствителен к пунктуации. Не использует морфологические метки. Параметры Bert модели открыты и можно попробовать их подредактировать.

In [ ]:
%%time
from deeppavlov import build_model, configs

sent_id = 0
model = build_model(configs.syntax.syntax_ru_syntagrus_bert, download=True)

sentences = ['С 31.01.11 г до 17.02.11 г находилась на лечении КБ № 81 гСеверска с диагнозом, Осн, ИБС, ОИМ в области нижней стенки левого желудочка, Q образующий от 28.01.2011 г']
for parse in model(sentences):
        print(parse, end="\n\n")

In [ ]:
%%time
from deeppavlov import build_model, configs

sent_id = 0
model = build_model("ru_syntagrus_joint_parsing", download=True)#(configs.syntax.syntax_ru_syntagrus_bert, download=True)
with open("DeepPavlov_7777.txt", "w") as f:
    for sent in range(len(sentences_7777)//3+1):
        try:
            for parse in model(sentences_7777[sent*3:(sent+1)*3]):
                print("# sent_id = " + str(sent_id), file=f)
                print("# text = " + sentences_7777[sent_id], file=f)
                print(parse, file=f)
                print(file=f)
                sent_id += 1
        except:
            pass

## Test

In [2]:
for sent in range(len(sentences_7777)):
    if sentences_7777[sent][-1] != '.':
        sentences_7777[sent] = sentences_7777[sent] + '.'

In [ ]:
%%time
from deeppavlov import build_model, configs

sent_id = 0
model = build_model(configs.syntax.syntax_ru_syntagrus_bert, download=True)
with open("DeepPavlov_test.txt", "w") as f:
    for sent in range(len(test_sentences)//3+1):
        try:
            for parse in model(test_sentences[sent*3:(sent+1)*3]):
                print("# sent_id = " + str(sent_id), file=f)
                print("# text = " + test_sentences[sent_id], file=f)
                print(parse, file=f)
                sent_id += 1
        except:
            pass

Time - 2 min 3s (GPU)

In [17]:
f = open('DeepPavlov_test.txt', 'r')
text = f.read()
text = text.replace("''", '"')
text = text.replace("``", '"')
f = open('DeepPavlov_test.txt', 'w')
f.write(text)

4178657

In [18]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu DeepPavlov_test.txt

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     99.32 |     98.43 |     98.88 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |     99.32 |     98.43 |     98.88 |
UPOS       |      0.00 |      0.00 |      0.00 |      0.00
XPOS       |     99.32 |     98.43 |     98.88 |    100.00
UFeats     |     37.82 |     37.49 |     37.65 |     38.08
AllTags    |      0.00 |      0.00 |      0.00 |      0.00
Lemmas     |      0.00 |      0.00 |      0.00 |      0.00
UAS        |     93.82 |     92.98 |     93.40 |     94.46
LAS        |     92.49 |     91.66 |     92.07 |     93.12
CLAS       |     91.85 |     91.04 |     91.45 |     92.50
MLAS       |      0.00 |      0.00 |      0.00 |      0.00
BLEX       |      0.00 |      0.00 |      0.00 |      0.00


* UAS - 93.40
* LAS - 92.07

# Slovnet

https://github.com/natasha/slovnet

Уже интегрирован в Natasha, но позволяет использовать выходной формат CONLL-U, а также можно попробовать другие настройки

## Установка необходимых библиотек

In [ ]:
!pip install slovnet

In [ ]:
!pip install Razdel

In [ ]:
!pip install navec

## Построение деревьев

In [ ]:
from ipymarkup import show_dep_markup as show_markup
from navec import Navec
from razdel import sentenize, tokenize
from slovnet import Syntax

navec = Navec.load("navec_news_v1_1B_250K_300d_100q.tar")
syntax = Syntax.load("slovnet_syntax_news_v1.tar")
syntax.navec(navec)

for sent in sentences:
    print(sent, end="\n\n")
    chunk = []
    tokens = [_.text for _ in tokenize(sent)]
    chunk.append(tokens)

    markup = next(syntax.map(chunk))
    words, deps = [], []
    for token in markup.tokens:
        words.append(token.text)
        source = int(token.head_id) - 1
        target = int(token.id) - 1
        if source > 0 and source != target:
            deps.append([source, target, token.rel])
    show_markup(words, deps)
    print("*" * 150)

In [ ]:
%%time
with open("Slovnet_7777.txt", "w") as f:
    for sent in range(len(sentences_7777)):
        chunk = []
        tokens = [_.text for _ in tokenize(sentences_7777[sent])]
        chunk.append(tokens)

        markup = next(syntax.map(chunk))
        
        print("# sent_id = " + str(sent), file=f)
        print("# text = " + sentences_7777[sent], file=f)
        for i in markup.as_json['tokens']:
            print('{}\t{}\t_\t_\t_\t_\t{}\t{}\t_\t_'.format(i['id'], i['text'], i['head_id'], i['rel']), file=f)
        print(file=f)

In [11]:
sentences = ['В апреле 2010 года проходила лечение в отд НРС, проводилась РЧ-аблация желудочковой экстрасистолии - успешно, на контрольном холтере за сутки была 1 экстрасистола.',
             '2 раза в день, эгилок 50 мг 2 раза в день.',
             'Около 5 лет назад выявлен СД 2 типа, подагра.',
             'Около 5 лет назад выявлен СД второго типа, подагра.',
             'В последствии, в 1995 году выявлен эписиндром, с 1995 года принимает бензонал 1 таб вечером ежедневно, последний приступ в 1997 году.',
             'В апреле 2009 госпитализация в отделение ОСН, где проведено КВГ, стенозирующего атеросклероза коронарных артерий не выявлено. ',
             'После операции через 1-2 месяца количество пароксизмов уменьшилось.',
             'Длительность приступов до 2 часов, частота до нескольких раз в месяц. ',
             'Длительность приступов до двух часов, частота до нескольких раз в месяц. ',
             'Последняя госпитализация в НРС в феврале 2010 года, проводилось СМ ЭКГ – усредненная ЧСС - 71 в минуту.',
             'В последние несколько месяцев отмечает нарастание одышки, появились жалобы на учащенное сердцебиение. ',
             'Язвенная болезнь, язва желудка, стадия ремиссии, последнее обострение 1993 год. ',
             'Последний раз ФГДС в 2008 году - рубцевание.',
             'Обследовалась по м/ж, на ЭХО-КГ выявлено расширение ЛП, по данным УЗИ - диффузные изменения ЩЖ, гормоны ЩЖ - в пределах нормы (от 11.2010). ',
             'У сестры - открытый Боталлов проток сестра - ГБ мать - ГБ, умерла от ОНМК в 71',
             'Больной себя считает в течении последних 15 лет, когда впервые стала отмечтаь повышение АД, эпизоды слабости, к рачарм не обращалась, считая, что данные проявления связаны с психоэмоциональными перегрузками около 10 лет назад стала отмечать появление давящих болей за грудиной, повышение АД до цифр 200\100 мм.рт.ст.',
             'Нарушение ритма в течении 4х лет.',
             'Нарушение ритма в течении 4 лет.',
             'Весной 2010 года проконсультирована аритмологом рекомендована имплантация ЭКС в НИИ Кардиологии. ',
            'Весной 2010 года проконсультирована аритмологом, рекомендована имплантация ЭКС в НИИ Кардиологии.']

In [12]:
%%time
with open("words_errors.conllu", "w") as f:
    for sent in range(len(sentences)):
        chunk = []
        tokens = [_.text for _ in tokenize(sentences[sent])]
        chunk.append(tokens)

        markup = next(syntax.map(chunk))
        
#         print("# sent_id = " + str(sent), file=f)
#         print("# text = " + sentences_7777[sent], file=f)
        for i in markup.as_json['tokens']:
            print('{}\t{}'.format(i['id'], i['text']), file=f)
        print(file=f)

CPU times: user 270 ms, sys: 614 ms, total: 884 ms
Wall time: 42.2 ms


## Test

In [ ]:
%%time
with open("Syntagrus_test.txt", "w") as f:
    for sent in range(len(test_sentences)):
        chunk = []
        tokens = [_.text for _ in tokenize(test_sentences[sent])]
        chunk.append(tokens)

        markup = next(syntax.map(chunk))
        
        print("# sent_id = " + str(sent), file=f)
        print("# text = " + test_sentences[sent], file=f)
        for i in markup.as_json['tokens']:
            print('{}\t{}\t_\t_\t_\t_\t{}\t{}\t_\t_'.format(i['id'], i['text'], i['head_id'], i['rel']), file=f)
        print(file=f)

Time - 9.19 s

In [11]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu Slovnet_test.txt

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     98.79 |     98.37 |     98.58 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |     98.79 |     98.37 |     98.58 |
UPOS       |      0.00 |      0.00 |      0.00 |      0.00
XPOS       |     98.79 |     98.37 |     98.58 |    100.00
UFeats     |     37.77 |     37.61 |     37.69 |     38.23
AllTags    |      0.00 |      0.00 |      0.00 |      0.00
Lemmas     |      0.00 |      0.00 |      0.00 |      0.00
UAS        |     80.07 |     79.73 |     79.90 |     81.05
LAS        |     75.50 |     75.18 |     75.34 |     76.43
CLAS       |     72.80 |     72.37 |     72.59 |     73.73
MLAS       |      0.00 |      0.00 |      0.00 |      0.00
BLEX       |      0.00 |      0.00 |      0.00 |      0.00


* UAS - 81.05
* LAS - 76.43

# UDPipe

https://github.com/TakeLab/spacy-udpipe

## Установка необходимых библиотек

In [ ]:
!pip install spacy-udpipe

## Построение деревьев

In [2]:
import spacy_udpipe
from nltk import Tree

nlp = spacy_udpipe.load_from_path(
    lang="ru",
    path="./russian-syntagrus-ud-2.5-191206.udpipe",
    meta={"description": "Syntagrus"},
)


def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [ ]:
%%time
for sent in sentences:
    doc = nlp(sent)
    print(sent, end="\n\n")
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]
    print('*'*150)

## Сохранение результата в формате Conll-U

In [34]:
from spacy_conll import ConllFormatter

nlp = spacy_udpipe.load_from_path(
    lang="ru",
    path="./russian-syntagrus-ud-2.5-191206.udpipe",
    meta={"description": "Syntagrus"},
)
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter)

In [ ]:
%%time
with open("Udpipe_7777.txt", "w") as f:
    for sent in range(len(sentences_7777)):
        doc = nlp(sentences_7777[sent])
        sent_id = str(docs[sent])
        print("# sent_id = " + sent_id[sent_id.rfind("/") + 1 :], file=f)
        print("# text = " + sentences_7777[sent], file=f)
        for i in range(len(doc)):
            token = doc[i]
            if token.dep_ =='root':
                dep = 0
            else: dep = token.head.i+1
            print('{}\t{}\t{}\t{}\t_\t_\t{}\t{}\t_\t_'.format(i+1, token.text, token.lemma_.strip(), token.pos_, dep, token.dep_), file=f)
        print(file=f)

## Test

In [ ]:
%%time
with open("UDPipe_test.txt", "w") as f:
    for sent in range(len(test_sentences)):
        doc = nlp(test_sentences[sent])

        print("# sent_id = " + str(sent), file=f)
        print("# text = " + test_sentences[sent], file=f)
        for i in range(len(doc)):
            token = doc[i]
            if token.dep_ =='root':
                dep = 0
            else: dep = token.head.i+1
            print('{}\t{}\t{}\t{}\t_\t_\t{}\t{}\t_\t_'.format(i+1, token.text, token.lemma_.strip(), token.pos_, dep, token.dep_), file=f)
        print(file=f)

Time - 1min 14s (GPU)

In [37]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu UDPipe_test.txt

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     99.56 |     99.63 |     99.60 |
Sentences  |     99.19 |     99.60 |     99.39 |
Words      |     99.56 |     99.63 |     99.60 |
UPOS       |     97.75 |     97.82 |     97.78 |     98.18
XPOS       |      0.00 |      0.00 |      0.00 |      0.00
UFeats     |     38.28 |     38.30 |     38.29 |     38.44
AllTags    |      0.00 |      0.00 |      0.00 |      0.00
Lemmas     |     96.51 |     96.58 |     96.55 |     96.94
UAS        |     87.65 |     87.71 |     87.68 |     88.03
LAS        |     80.03 |     80.09 |     80.06 |     80.38
CLAS       |     82.09 |     74.76 |     78.25 |     75.09
MLAS       |      7.19 |      6.55 |      6.86 |      6.58
BLEX       |     79.06 |     71.99 |     75.36 |     72.31


* UAS - 87.68
* LAS - 80.06

# Stanza (formerly StanfordNLP)

https://stanfordnlp.github.io/stanza/#getting-started

## Установка необходимых библиотек

In [ ]:
!pip install spacy-stanza

In [ ]:
import stanza
stanza.download('ru') # syntagrus is default model for Stanza

## Построение деревьев

In [2]:
import stanza
from nltk import Tree
from spacy_stanza import StanzaLanguage

snlp = stanza.Pipeline(lang="ru")
nlp = StanzaLanguage(snlp)


def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

2020-08-10 11:22:17 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2020-08-10 11:22:17 INFO: Use device: gpu
2020-08-10 11:22:17 INFO: Loading: tokenize
2020-08-10 11:22:19 INFO: Loading: pos
2020-08-10 11:22:20 INFO: Loading: lemma
2020-08-10 11:22:20 INFO: Loading: depparse
2020-08-10 11:22:21 INFO: Loading: ner
2020-08-10 11:22:22 INFO: Done loading processors!


In [ ]:
%%time
for sent in sentences:
    doc = nlp(sent)
    print(sent, end="\n\n")
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]
    print('*'*150)

## Сохранение результата в формате Conll-U

In [3]:
from spacy_conll import ConllFormatter

snlp = stanza.Pipeline(lang="ru")
nlp = StanzaLanguage(snlp)

2020-08-10 11:22:23 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2020-08-10 11:22:23 INFO: Use device: gpu
2020-08-10 11:22:23 INFO: Loading: tokenize
2020-08-10 11:22:23 INFO: Loading: pos
2020-08-10 11:22:24 INFO: Loading: lemma
2020-08-10 11:22:24 INFO: Loading: depparse
2020-08-10 11:22:24 INFO: Loading: ner
2020-08-10 11:22:25 INFO: Done loading processors!


In [4]:
%%time
with open("Stanza_100.txt", "w") as f:
    for sent in range(len(sentences_7777))[:100]:
        if sentences_7777[sent][-1] != '.':
            sentences_7777[sent] = sentences_7777[sent] + '.'
        doc = nlp(sentences_7777[sent])
        sent_id = str(docs[sent])
        print("# sent_id = " + sent_id[sent_id.rfind("/") + 1 :], file=f)
        print("# text = " + sentences_7777[sent], file=f)
        for i in range(len(doc)):
            token = doc[i]
            if token.dep_ =='root':
                dep = 0
            else: dep = token.head.i+1
            print('{}\t{}\t{}\t{}\t_\t_\t{}\t{}\t_\t_'.format(i+1, token.text, token.lemma_.strip(), token.pos_, dep, token.dep_), file=f)
        print(file=f)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, CPU times: user 4.55 s, sys: 21.5 ms, total: 4.57 s
Wall time: 4.59 s


## Test

In [ ]:
%%time
with open("Stanza_test.txt", "w") as f:
    for sent in range(len(test_sentences)):
        doc = nlp(test_sentences[sent])

        print("# sent_id = " + str(sent), file=f)
        print("# text = " + test_sentences[sent], file=f)
        print(doc._.conll_str, file=f)

Time - 5min 53s (GPU)

In [25]:
!python conll18_ud_eval.py -v ru_syntagrus-ud-test.conllu Stanza_test.txt

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |     99.29 |     99.26 |     99.28 |
Sentences  |     91.65 |     95.66 |     93.61 |
Words      |     99.29 |     99.26 |     99.28 |
UPOS       |     97.91 |     97.89 |     97.90 |     98.62
XPOS       |      0.00 |      0.00 |      0.00 |      0.00
UFeats     |     38.11 |     38.10 |     38.11 |     38.39
AllTags    |      0.00 |      0.00 |      0.00 |      0.00
Lemmas     |     97.23 |     97.21 |     97.22 |     97.93
UAS        |     91.76 |     91.74 |     91.75 |     92.42
LAS        |     89.98 |     89.95 |     89.97 |     90.62
CLAS       |     88.82 |     88.62 |     88.72 |     89.30
MLAS       |      6.90 |      6.88 |      6.89 |      6.93
BLEX       |     86.59 |     86.39 |     86.49 |     87.05


* UAS - 91.75
* LAS - 89.97